# Fetch U.S. Daily Energy Demand from EIA API

This notebook fetches hourly electricity demand data from the U.S. Energy Information Administration (EIA) API and aggregates it to daily demand. Replace `YOUR_API_KEY` with your actual EIA API key.

In [ ]:
# Install required libraries if not already installed
!pip install requests pandas matplotlib

In [ ]:
import requests
import pandas as pd
import os

# Set your EIA API key
API_KEY = "YOUR_API_KEY"  # Replace with your actual API key or use os.getenv("EIA_API_KEY")
BASE_URL = "https://api.eia.gov/v2/"
ENDPOINT = "electricity/rto/region-sub-ba-data/data/"

# Construct API request
url = f"{BASE_URL}{ENDPOINT}"
params = {
    "api_key": API_KEY,
    "frequency": "hourly",
    "data[0]": "value",
    "start": "2025-01-01",
    "end": "2025-07-20",
    "sort[0][column]": "period",
    "sort[0][direction]": "asc",
    "offset": 0,
    "length": 5000
}

# Fetch data
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    if 'response' in data and 'data' in data['response']:
        # Convert to DataFrame
        df = pd.DataFrame(data['response']['data'])
        
        # Convert 'period' to datetime and aggregate to daily
        df['period'] = pd.to_datetime(df['period'])
        df_daily = df.groupby(df['period'].dt.date)['value'].sum().reset_index()
        df_daily.columns = ['date', 'daily_demand_mwh']
        
        # Display first few rows
        print(df_daily.head())
        
        # Save to CSV
        df_daily.to_csv('us_daily_energy_demand.csv', index=False)
    else:
        print("Error in response:", data.get('error', 'Unknown error'))
else:
    print(f"Request failed with status code: {response.status_code}")

: 

In [ ]:
# Visualize the data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df_daily['date'], df_daily['daily_demand_mwh'], marker='o')
plt.title('U.S. Daily Electricity Demand (2025)')
plt.xlabel('Date')
plt.ylabel('Demand (MWh)')
plt.grid(True)
plt.show()

## Notes
- Replace `YOUR_API_KEY` with your actual EIA API key.
- The script fetches up to 5,000 rows. For more data, modify the `offset` parameter in a loop.
- To filter by region, add `facets[parent][]=` (e.g., `CISO` for California ISO) to the `params` dictionary.
- Data is saved to `us_daily_energy_demand.csv` in your working directory.